In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd
import time


chrome_options = Options()

service = Service("D:/chromedriver-win64/chromedriver-win64/chromedriver.exe") 
driver = webdriver.Chrome(service=service, options=chrome_options)


base_url = 'https://store.steampowered.com/search/?filter=topsellers'
columns_ = ['Names', 'Links', 'Price', 'Release Date', 'Requirements']
df = pd.DataFrame(columns=columns_)

try:
    all_links = []
    for i in range(1, 358): 
        url = f"{base_url}&page={i}"
        driver.get(url)
        time.sleep(3) 

        soup = BeautifulSoup(driver.page_source, 'html.parser')

        search_results = soup.find_all('a', {'class': 'search_result_row'})

        for result in search_results:
            link = result.get('href')
            if link:
                all_links.append(link)

    new_rows = []
    for link in all_links:
        try:
            driver.get(link)
            time.sleep(3) 

            soup = BeautifulSoup(driver.page_source, 'html.parser')

            name = soup.find('span', {'class': 'title'})
            name_text = name.text.strip() if name else 'No Name'

            price_div = soup.find('div', {'class': 'discount_final_price'})
            price_text = price_div.text.strip() if price_div else 'No Price'

            release_date = soup.find('div', {'class': 'date'})
            release_text = release_date.text.strip() if release_date else 'No Release Date'

            requirements_ul = soup.find('ul', class_='bb_ul')
            requirements_text = [li.get_text(strip=True) for li in requirements_ul.find_all('li')] if requirements_ul else ['No Requirements']

            requirements_text_combined = ' | '.join(requirements_text)

            new_rows.append({
                'Names': name_text,
                'Links': link,
                'Price': price_text,
                'Release Date': release_text,
                'Requirements': requirements_text_combined
            })

        except Exception as e:
            print(f"Error processing link {link}: {e}")

    df = pd.DataFrame(new_rows)

finally:
    driver.quit()

print("Final DataFrame with data from all pages:")
print(df)


Final DataFrame with data from all pages:
       Names                                              Links  \
0    No Name  https://store.steampowered.com/app/2180340/Mir...   
1    No Name  https://store.steampowered.com/app/2285964/FOR...   
2    No Name  https://store.steampowered.com/app/3181440/Kit...   
3    No Name  https://store.steampowered.com/app/2693380/Sur...   
4    No Name  https://store.steampowered.com/app/334940/Yoku...   
..       ...                                                ...   
245  No Name  https://store.steampowered.com/app/1001291/Wor...   
246  No Name  https://store.steampowered.com/app/626470/Town...   
247  No Name  https://store.steampowered.com/app/1352850/Cit...   
248  No Name  https://store.steampowered.com/app/2916190/Clo...   
249  No Name  https://store.steampowered.com/app/515960/Mart...   

                    Price     Release Date  \
0                34.38 SR      2 Jun, 2024   
1                No Price     16 Mar, 2023   
2              

In [2]:
df.to_csv('game.csv')